# Imports

In [120]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score,classification_report\

import joblib

In [114]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Data & Setup

In [41]:
df = pd.read_csv('./Sleep Disorders/Sleep_health_and_lifestyle_dataset.csv')

In [44]:
df = df.set_index('Person ID')

In [61]:
df.isnull().sum()

Gender                       0
Age                          0
Occupation                   0
Sleep Duration               0
Quality of Sleep             0
Physical Activity Level      0
Stress Level                 0
BMI Category                 0
Blood Pressure               0
Heart Rate                   0
Daily Steps                  0
Sleep Disorder             219
dtype: int64

In [62]:
df['Sleep Disorder'].unique()

array([nan, 'Sleep Apnea', 'Insomnia'], dtype=object)

### Putting NaN values as not any disorders

In [72]:
def func(df):
    if df in ['Sleep Apnea','Insomnia']:
        return df
    else:
        return 'No Disorders'

In [74]:
df['Sleep Disorder'] = df['Sleep Disorder'].apply(func)

In [76]:
X = df.drop('Sleep Disorder',axis = 1)
y = df['Sleep Disorder']

In [79]:
X = pd.get_dummies(X)

## Data Split

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# 1) Multi-Layer Perceptron
   ### 100 x 100 x 100 hidden layers

In [8]:
mlp = MLPClassifier(hidden_layer_sizes=[100,100,100],random_state=42,
                   max_iter =500, early_stopping = True)

In [110]:
mlp.fit(X_train,y_train)

MLPClassifier(early_stopping=True, hidden_layer_sizes=[100, 100, 100],
              max_iter=500, random_state=42)

# Results

In [111]:
ypred1 = mlp.predict(X_test)
accuracy_score(y_test,ypred1)

0.5733333333333334

In [116]:
print(classification_report(y_test,ypred1))

              precision    recall  f1-score   support

    Insomnia       0.00      0.00      0.00        16
No Disorders       0.57      1.00      0.73        43
 Sleep Apnea       0.00      0.00      0.00        16

    accuracy                           0.57        75
   macro avg       0.19      0.33      0.24        75
weighted avg       0.33      0.57      0.42        75



# 2) PipeLine 
 with Data Normalization

In [9]:
pipe = make_pipeline(StandardScaler(),mlp)

In [81]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('mlpclassifier',
                 MLPClassifier(early_stopping=True,
                               hidden_layer_sizes=[100, 100, 100], max_iter=500,
                               random_state=42))])

In [82]:
ypred = pipe.predict(X_test)

In [83]:
accuracy_score(y_test,ypred)

0.88

In [84]:
print(classification_report(y_test,ypred))

              precision    recall  f1-score   support

    Insomnia       0.82      0.88      0.85        16
No Disorders       0.89      0.98      0.93        43
 Sleep Apnea       0.91      0.62      0.74        16

    accuracy                           0.88        75
   macro avg       0.88      0.83      0.84        75
weighted avg       0.88      0.88      0.87        75



### 3) Hyper-parameter Tuning

In [104]:
mlp1 = MLPClassifier(hidden_layer_sizes=[100,100,100,100,100],random_state=42,
                   max_iter =5000, early_stopping = True,
                    beta_1=0.8,beta_2=0.7,epsilon=1e-7,
                    n_iter_no_change=100,learning_rate_init=0.01,
                    validation_fraction=0.3)

In [105]:
pipe1 = make_pipeline(StandardScaler(),mlp1)

In [106]:
pipe1.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('mlpclassifier',
                 MLPClassifier(beta_1=0.8, beta_2=0.7, early_stopping=True,
                               epsilon=1e-07,
                               hidden_layer_sizes=[100, 100, 100, 100, 100],
                               learning_rate_init=0.01, max_iter=5000,
                               n_iter_no_change=100, random_state=42,
                               validation_fraction=0.3))])

In [107]:
ypred01 = pipe1.predict(X_test)

In [108]:
accuracy_score(y_test,ypred01)

0.88

In [109]:
print(classification_report(y_test,ypred01))

              precision    recall  f1-score   support

    Insomnia       0.76      0.81      0.79        16
No Disorders       0.91      0.98      0.94        43
 Sleep Apnea       0.92      0.69      0.79        16

    accuracy                           0.88        75
   macro avg       0.86      0.83      0.84        75
weighted avg       0.88      0.88      0.88        75



# Final Model:-
>   ## 2nd Model

In [121]:
joblib.dump(pipe,'MLP_Pipeline.pkl')

['MLP_Pipeline.pkl']